# Clasificación de municipios (Similares)

Un municipio puede ser “similar” a otro según varias dimensiones, como puede ser su Índice global (aunque esta es demasiado general). En este caso se emplean los vectores de categorías (salud, educación, bienestar, economía…). Municipios similares serán los que tengan perfiles parecidos en estas dimensiones.

Los motivos son los siguientes:

- Están normalizadas (0–1).

- Resumen información real del municipio.

- Son robustas a ruido.

- Son interpretables (cuando veas un cluster sabrás por qué están juntos).

Así que el clustering se hará sobre estas 6 columnas.

In [1]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_distances

In [2]:
# Cargar el conjunto de datos
df = pd.read_csv("website_data/df_indices_categorias.csv")

feature_cols = ["sanidad", "educacion", "transporte_conectividad", "economia_empleo", "bienestar", "ocio_cultura"]
X = df[feature_cols].copy()
municipios = df["municipio"]

print(X.head())

    sanidad  educacion  transporte_conectividad  economia_empleo  bienestar  \
0  0.078298   0.785864                 0.201663         0.616619   0.819270   
1  0.664198   0.594422                 0.921814         0.598718   0.792125   
2  0.670480   0.554393                 0.198153         0.425267   0.790199   
3  0.634423   0.538964                 0.642718         0.438277   0.767845   
4  0.643804   0.493591                 0.425092         0.350587   0.829896   

   ocio_cultura  
0      0.176919  
1      0.065626  
2      0.155993  
3      0.090764  
4      0.105947  


## PCA

- No dependes de clusters.
- Cada municipio tendrá su propio conjunto único de similares,

In [3]:
# PCA (2 O 3 componentes funcionanmuy bien)
pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

# Matriz de distancias
dist_matrix = cosine_distances(X_pca)

# Función para obtener los municipios más similares
k = 10  # Número de municipios similares a encontrar

similares = {}

for i, muni in enumerate(municipios):
    # ordenar por distancia (de menor a mayor)
    idx = dist_matrix[i].argsort()[1:k+1]
    similares[muni] = municipios.iloc[idx].tolist()

# Convertimos a tabla
df_similares = (
    pd.DataFrame.from_dict(similares, orient="index")
      .reset_index()
)

df_similares.columns = ["municipio", "sim1", "sim2", "sim3", "sim4", "sim5", "sim6", "sim7", "sim8", "sim9", "sim10"]

df_similares.to_csv("website_data/municipios_similares.csv", index=False)
df_similares.head()

,municipio,sim1,sim2,sim3,sim4,sim5,sim6,sim7,sim8,sim9,sim10
0,La Acebeda,Gascones,Horcajuelo de la Sierra,Piñuécar-Gandullas,La Serna del Monte,Madarcos,Prádena del Rincón,Somosierra,Horcajo de la Sierra,Robledillo de la Jara,Cervera de Buitrago
1,Ajalvir,Colmenarejo,Villanueva del Pardillo,Cobeña,Collado Mediano,Loeches,Titulcia,Algete,Hoyo de Manzanares,Torrelodones,Brunete
2,Alameda del Valle,Pelayos de la Presa,San Martín de Valdeiglesias,Rascafría,Pinilla del Valle,Patones,Garganta de los Montes,Canencia,Valdemaqueda,Robledo de Chavela,Fresnedillas de la Oliva
3,El Álamo,Quijorna,Casarrubuelos,Valdemorillo,Valdetorres de Jarama,Villar del Olmo,Talamanca de Jarama,El Escorial,Villanueva de la Cañada,Batres,Serranillos del Valle
4,Aldea del Fresno,Navas del Rey,Torrelaguna,Olmeda de las Fuentes,Orusco de Tajuña,Colmenar del Arroyo,La Cabrera,Belmonte de Tajo,Valdemanco,Guadalix de la Sierra,Ambite


¿Por qué usar distancia coseno?

Porque es más estable cuando los datos están estandarizados.

Evita que un municipio con valores más grandes parezca “más lejano”

Captura muy bien el “perfil” de un municipio